<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Description" data-toc-modified-id="Data-Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Description</a></span></li><li><span><a href="#Useful-Scripts" data-toc-modified-id="Useful-Scripts-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Useful Scripts</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Memory-Reduction" data-toc-modified-id="Memory-Reduction-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Memory Reduction</a></span></li><li><span><a href="#Exploratory-Data-Analysis-(EDA)" data-toc-modified-id="Exploratory-Data-Analysis-(EDA)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Exploratory Data Analysis (EDA)</a></span><ul class="toc-item"><li><span><a href="#Top-5-pages-per-language" data-toc-modified-id="Top-5-pages-per-language-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Top 5 pages per language</a></span></li></ul></li><li><span><a href="#Data-Visualizations" data-toc-modified-id="Data-Visualizations-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Data Visualizations</a></span><ul class="toc-item"><li><span><a href="#Language-montly-mean" data-toc-modified-id="Language-montly-mean-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Language montly mean</a></span></li><li><span><a href="#Timeseries-per-language" data-toc-modified-id="Timeseries-per-language-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Timeseries per language</a></span></li><li><span><a href="#Fast-Fourier-Transform-(FFT)" data-toc-modified-id="Fast-Fourier-Transform-(FFT)-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Fast Fourier Transform (FFT)</a></span></li></ul></li></ul></div>

# Data Description

Reference: https://www.kaggle.com/c/web-traffic-time-series-forecasting/data

I have cleaned the kaggle wikipedia traffic data and selected only data of 2016 with 
fraction of 0.1.

The data was melted and additional columns were created.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set(context='notebook', style='whitegrid', rc={'figure.figsize': (12,8)})
plt.style.use('fivethirtyeight') # better than sns styles.
matplotlib.rcParams['figure.figsize'] = 12,8

import os
import time

# random state
random_state=100
np.random.seed(random_state)

# Jupyter notebook settings for pandas
#pd.set_option('display.float_format', '{:,.2g}'.format) # numbers sep by comma
from pandas.api.types import CategoricalDtype
np.set_printoptions(precision=3)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100) # None for all the rows
pd.set_option('display.max_colwidth', 200)

import IPython
from IPython.display import display, HTML, Image, Markdown

print([(x.__name__,x.__version__) for x in [np, pd,sns,matplotlib]])

[('numpy', '1.16.4'), ('pandas', '0.25.0'), ('seaborn', '0.9.0'), ('matplotlib', '3.1.1')]


In [2]:
import dask
import dask.dataframe as dd
import gc

In [3]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

# Useful Scripts

In [4]:
def show_method_attributes(method, ncols=7,start=None):
    """ Show all the attributes of a given method.
    Example:
    ========
    show_method_attributes(list)
     """
    x = [I for I in dir(method) if I[0]!='_' ]
    x = [I for I in x 
         if I not in 'os np pd sys time psycopg2'.split() ]
    if start:
        x = [I for I in x if I.startswith(start)]

    return pd.DataFrame(np.array_split(x,ncols)).T.fillna('')

# Load the data

In [32]:
df = pd.read_csv('../../data/wiki/processed/data_cleaned_2016_frac01.csv',
                 parse_dates=['date'])

print(df.shape) # 5.3 million rows, 21 cols
df.head()

(5309196, 21)


,Page,date,visits,year,month,day,quarter,dayofweek,dayofyear,day_name,month_name,weekend,weekday,mean,median,name,project,access,agent,lang,language
0,Sean_Connery_en.wikipedia.org_desktop_all-agents,2016-01-01,4872,2016,1,1,1,4,1,Friday,January,False,True,3405.661202,2624.0,Sean_Connery,en.wikipedia.org,desktop,all-agents,en,English
1,Tableau_des_médailles_des_Jeux_olympiques_d'été_de_2008_fr.wikipedia.org_desktop_all-agents,2016-01-01,6,2016,1,1,1,4,1,Friday,January,False,True,170.841530,18.0,Tableau_des_médailles_des_Jeux_olympiques_d'été_de_2008,fr.wikipedia.org,desktop,all-agents,fr,French
2,The_Undertaker_fr.wikipedia.org_mobile-web_all-agents,2016-01-01,469,2016,1,1,1,4,1,Friday,January,False,True,400.336066,345.5,The_Undertaker,fr.wikipedia.org,mobile-web,all-agents,fr,French
3,Category:Outdoor_sex_commons.wikimedia.org_all-access_all-agents,2016-01-01,142,2016,1,1,1,4,1,Friday,January,False,True,205.174863,193.0,Category:Outdoor_sex,commons.wikimedia.org,all-access,all-agents,commons,Media
4,Камызяк_ru.wikipedia.org_all-access_all-agents,2016-01-01,6692,2016,1,1,1,4,1,Friday,January,False,True,912.516393,559.0,Камызяк,ru.wikipedia.org,all-access,all-agents,ru,Russian


# Memory Reduction

In [33]:
df.dtypes

Page                  object
date          datetime64[ns]
visits                 int64
year                   int64
month                  int64
day                    int64
quarter                int64
dayofweek              int64
dayofyear              int64
day_name              object
month_name            object
weekend                 bool
weekday                 bool
mean                 float64
median               float64
name                  object
project               object
access                object
agent                 object
lang                  object
language              object
dtype: object

In [34]:
df.memory_usage(deep=True).sum() * 1e-6 # MB

4069.15316

In [35]:
# all the year is 2016,drop it.

df.drop('year',axis=1,inplace=True)

In [36]:
cols_int = ['visits']
cols_cat = ['month','day','quarter','day_name','month_name',
            'project','access','agent','language']

cols_float = ['mean','median']

for c in cols_int:
    df[c] = df[c].astype(np.int32)
    
for c in cols_float:
    df[c] = df[c].astype(np.float32)
    

for c in cols_cat:
    df[c] = df[c].astype(pd.api.types.CategoricalDtype())

In [37]:
df.memory_usage(deep=True).sum() * 1e-6 # MB

1777.2233549999999